# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

D. Haegele  ->  D. Haegele  |  ['D. Haegele']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
C. Gieser  ->  C. Gieser  |  ['C. Gieser']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 71 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.21588


extracting tarball to tmp_2509.21588... done.


/tmp/ipykernel_3530/2822249172.py:52: LatexWarning: 2509.21588 did not run properly
bad escape \i at position 8
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2509.21643


...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.22541


extracting tarball to tmp_2509.22541...

 done.


Found 272 bibliographic references in tmp_2509.22541/sn-article_revised.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2509.22602


extracting tarball to tmp_2509.22602... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.21643-b31b1b.svg)](https://arxiv.org/abs/2509.21643) | **A nearly pristine star from the Large Magellanic Cloud**  |
|| A. P. Ji, et al. -- incl., <mark>P. Eitner</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *18 pages, 9 figures, 1 table, submitted*|
|**Abstract**|            The first stars formed out of pristine gas, causing them to be so massive that none are expected to have survived until today. If their direct descendants were sufficiently low-mass stars, they could exist today and would be recognizable by having the lowest metallicity (abundance of elements heavier than helium). The lowest metallicity star currently known is a star in the thick disk of the Milky Way with total metallicity Z < 1.4 x 10^-6 (log Z/Zsun < -4.0). While other stars with lower iron abundance have been discovered, they have high carbon abundances and thus higher total metallicities (log Z/Zsun > -3). Here we present the discovery and detailed chemical analysis of the most metal-poor star yet found: the red giant star SDSS J0715-7334 with ultra-low abundances of both iron and carbon ([Fe/H]=-4.3, [C/Fe]<-0.2), resulting in total metallicity Z < 7.8 x 10^-7 (log Z/Zsun < -4.3). This star has the most pristine composition of any object known in the universe. The star's orbit indicates that it originates from the halo of the Large Magellanic Cloud. Its detailed chemical composition implies a supernova progenitor with initial mass of 30 solar masses. Current models of low-mass star formation can explain the existence of SDSS J0715-7334 only if dust cooling was already able to operate at the time of its formation. SDSS J0715-7334 is over ten times more metal-poor than the most metal-poor high-redshift galaxies found by the James Webb Space Telescope, some of which have been claimed to be potentially metal-free. Substantially deeper observations of high-redshift galaxies would be needed to prove that they are truly pristine galaxies made of metal-free stars and not metal-enriched galaxies composed of second-generation stars like SDSS J0715-7334.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.22541-b31b1b.svg)](https://arxiv.org/abs/2509.22541) | **The Origins & Reservoirs of Exocomets**  |
|| M. Bannister, et al. -- incl., <mark>H. Klahr</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *Accepted for publication in Space Science Reviews. Chapter written for the the workshop 'Exocomets: Bridging our Understanding of Minor Bodies in Solar and Exoplanetary Systems' (July 2024), at the International Space Science Insitute (ISSI), Bern*|
|**Abstract**|            Small bodies exist in distinct populations within their planetary systems. These reservoir populations hold a range of compositions, which to first order are dependent on formation location relative to their star. We provide a general overview of the nature of the reservoirs that source exocomets, from the influence of the stellar environment through planetesimal formation to comparisons with Solar System populations. Once transitioned from a young protoplanetary disc to a debris disc, a star can expect to be rained with exocomets. While exocomets are predominantly detected to date at A-type stars, planetesimals plausibly exist across a range of stellar masses, based on exoplanet abundance, debris disc occurrence and white dwarf infall.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.22179-b31b1b.svg)](https://arxiv.org/abs/2509.22179) | **The impact of cosmic filaments on the abundance of satellite galaxies**  |
|| Y. Meng, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *16 pages, 7 figures; Submitted to ApJ*|
|**Abstract**|            The impact of cosmic web environments on galaxy properties plays a critical role in understanding galaxy formation. Using the state-of-the-art cosmological simulation IllustrisTNG, we investigate how satellite galaxy abundance differs between filaments and the field, with filaments identified using the DisPerSE algorithm. When filaments are identified using galaxies as tracers, we find that, across all magnitude bins, central galaxies in filaments tend to host more satellite galaxies than their counterparts in the field, in qualitative agreement with observational results from the Sloan Digital Sky Survey. The average ratios between satellite luminosity functions in filaments and the field are $3.49$, $2.61$, and $1.90$ in the central galaxy $r$-band magnitude bins of $M_{r, {\rm cen}} \sim -22$, $-21$, and $-20$, respectively. We show that much of this excess can be attributed to the higher host halo masses of galaxies in filaments. After resampling central galaxies in both environments to match the halo mass distributions within each magnitude bin, the satellite abundance enhancement in filaments is reduced by up to $79 \%$. Additionally, the choice of tracers used to identify filaments introduces a significant bias: when filaments are identified using the dark matter density field, the environmental difference in satellite abundance is reduced by more than $70 \%$; after further resampling in both magnitude and halo mass, the difference is further suppressed by another $\sim 60$--$95 \%$. Our results highlight the importance of halo mass differences and tracer choice biases when interpreting and understanding the impact of environment on satellite galaxy properties.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.22382-b31b1b.svg)](https://arxiv.org/abs/2509.22382) | **The Simons Observatory: Characterization of the 220/280 GHz TES Detector Modules**  |
|| D. Dutcher, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *6 pages, 7 figures. Proceedings of the 21st International Conference on Low Temperature Detectors (LTD2025)*|
|**Abstract**|            The Simons Observatory (SO) is a new suite of cosmic microwave background telescopes in the Chilean Atacama Desert with an extensive science program spanning cosmology, Galactic and extragalactic astrophysics, and particle physics. SO will survey the millimeter-wave sky over a wide range of angular scales using six spectral bands across three types of dichroic, polarization-sensitive transition-edge sensor (TES) detector modules: Low-Frequency (LF) modules with bandpasses centered near 30 and 40 GHz, Mid-Frequency (MF) modules near 90 and 150 GHz, and Ultra-High-Frequency (UHF) modules near 220 and 280 GHz. Twenty-five UHF detector modules, each containing 1720 optically-coupled TESs connected to microwave SQUID multiplexing readout, have now been produced. This work summarizes the pre-deployment characterization of these detector modules in laboratory cryostats. Across all UHF modules, we find an average operable TES yield of 83%, equating to over 36,000 devices tested. The distributions of (220, 280) GHz saturation powers have medians of (24, 26) pW, near the centers of their target ranges. For both bands, the median optical efficiency is 0.6, the median effective time constant is 0.4 ms, and the median dark noise-equivalent power (NEP) is ~40 aW/rtHz. The expected photon NEPs at (220, 280) GHz are (64, 99) aW/rtHz, indicating these detectors will achieve background-limited performance on the sky. Thirty-nine UHF and MF detector modules are currently operating in fielded SO instruments, which are transitioning from the commissioning stage to full science observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.22602-b31b1b.svg)](https://arxiv.org/abs/2509.22602) | **Small Near-Earth Objects in the Taurid Resonant Swarm**  |
|| Q. Ye, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *Accepted for publication on Acta Astronautica, PDC 2025 Special Issue*|
|**Abstract**|            The Taurid Resonant Swarm (TRS) within the Taurid Complex hosts dynamically-concentrated debris in a 7:2 mean-motion resonance with Jupiter. Fireball observations have confirmed that the TRS is rich in sub-meter-sized particles, but whether this enhancement extends to larger, asteroid-sized objects remains unclear. Here we reanalyze the data obtained by a Zwicky Transient Facility (ZTF) campaign during the 2022 TRS encounter, and find that the TRS may host up to $\sim10^2$ Tunguska-sized objects and up to $\sim10^3$ Chelyabinsk-sized objects, the latter of which agrees the estimate derived from bolide records. This translates to an impact frequency of less than once every 4 million years. However, we caution that these numbers are based on the unverified assumption that the orbital distribution of the TRS asteroids follows that of fireball-sized meteoroids. Future wide-field facilities, such as the Vera C. Rubin Observatory, could take advantage of TRS's close approaches in the 2020-30s and validate the constraints of the asteroid-sized objects in the TRS.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.21588-b31b1b.svg)](https://arxiv.org/abs/2509.21588) | **Limb Asymmetries on WASP-39b: A Multi-GCM Comparison of Chemistry, Clouds, and Hazes**  |
|| M. E. Steinrueck, et al. -- incl., <mark>D. Haegele</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *submitted to AAS Journals. Supplementary Data available on Zenodo*|
|**Abstract**|            With JWST, observing separate spectra of the morning and evening limbs of hot Jupiters has finally become a reality. The first such observation was reported for WASP-39b, where the evening terminator was observed to have a larger transit radius by about 400 ppm and a stronger 4.3 $\mu$m CO$_2$ feature than the morning terminator. Multiple factors, including temperature differences, photo/thermochemistry, clouds and hazes, could cause such limb asymmetries. To interpret these new limb asymmetry observations, a detailed understanding of how the relevant processes affect morning and evening spectra grounded in forward models is needed. Focusing on WASP-39b, we compare simulations from five different general circulation models (GCMs), including one simulating disequilibrium thermochemistry and one with cloud radiative feedback, to the recent WASP-39b limb asymmetry observations. We also post-process the temperature structures of all simulations with a 2D photochemical model and one simulation with a cloud microphysics model. Although the temperatures predicted by the different models vary considerably, the models are remarkably consistent in their predicted morning--evening temperature differences. Several equilibrium-chemistry simulations predict strong methane features in the morning spectrum, not seen in the observations. When including disequilibrium processes, horizontal transport homogenizes methane, and these methane features disappear. However, even after including photochemistry and clouds, our models still cannot reproduce the observed ${\sim}2000$ ppm asymmetry in the CO$_2$ feature. A combination of factors, such as varying metallicity and unexplored parameters in cloud models, may explain the discrepancy, emphasizing the need for future models integrating cloud microphysics and feedback across a broader parameter space.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \i at position 8</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.21701-b31b1b.svg)](https://arxiv.org/abs/2509.21701) | **First results from ALPPS: a sub-Alfvénic streamer in SVS13A**  |
|| P. C. Cortes, et al. -- incl., <mark>C. Gieser</mark> |
|*Appeared on*| *2025-09-29*|
|*Comments*| *Accepted for publication is Astrophiscal Journal Letters*|
|**Abstract**|            We present the first results from the ALMA Perseus Polarization Survey (ALPPS), focusing on the magnetic field in the SVS13A circumbinary disk. The dataset includes full-Stokes dust continuum observations at $\sim0\farcs3$ and 870 $\mu$m, as well as molecular line emission from C$^{17}$O$(J=3 \rightarrow 2)$ at $\sim0\farcs3$, C$^{18}$O$(J=2 \rightarrow 1)$ at $\sim0\farcs2$, and DCN$(J=3 \rightarrow 2)$ at $\sim0\farcs1$ angular resolution. Our observations resolve both a previously identified dust spiral and an infalling streamer, capturing their spatial and kinematic structures. The streamer is traced from scales $>300$ au down to the circumbinary disk. Using alignment measure (AM) maps and histograms that compare the orientations of the plane-of-sky magnetic field with local intensity and velocity gradients, we find that the AM distribution peaks at a value of 1. This AM peak strongly suggests alignment between the field and the dust total intensity emission, as well as between the field and the gas velocity, which in turn suggests grain alignment by magnetic fields. From our data, we derive a magnetic field strength, B$_{\mathrm{pos}} \sim 1.1 \pm 0.6$\, mG, and a kinetic to magnetic energy ratio of $0.5 \pm 0.4$, suggesting magnetic dominance. We also produced a map of the Alfvénic Mach number, finding $\mathcal{M}_{\rm A} < 1$ along the streamer, consistent with sub-Alfvénic infalling motions. Therefore, the field is likely facilitating the inflow of material from the envelope onto the disk by constraining movement across the field lines. This represents the first detection of a magnetically sub-Alfvénic infalling streamer in a protostellar system.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error string index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.21643/./nature_2a_fehch.png', 'tmp_2509.21643/./ump_yz_timeticks.png', 'tmp_2509.21643/./ump_orbit_map.png', 'tmp_2509.21643/./nature_pop3fit.png']
copying  tmp_2509.21643/./nature_2a_fehch.png to _build/html/
copying  tmp_2509.21643/./ump_yz_timeticks.png to _build/html/
copying  tmp_2509.21643/./ump_orbit_map.png to _build/html/
copying  tmp_2509.21643/./nature_pop3fit.png to _build/html/
exported in  _build/html/2509.21643.md
    + _build/html/tmp_2509.21643/./nature_2a_fehch.png
    + _build/html/tmp_2509.21643/./ump_yz_timeticks.png
    + _build/html/tmp_2509.21643/./ump_orbit_map.png
    + _build/html/tmp_2509.21643/./nature_pop3fit.png
found figures ['tmp_2509.22541/./mDustAndWidthVsAge.png', 'tmp_2509.22541/./ExocometsInterstellar.jpg', 'tmp_2509.22541/./Exocomet_Growth.png']
copying  tmp_2509.22541/./mDustAndWidthVsAge.png to _build/html/
copying  tmp_2509.22541/./ExocometsInterstellar.jpg to _build/html/
copying  tmp_2509.22541/./Exocomet_Growth.png t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\pasa}{PASA}$
$\newcommand{\pasp}{PASP}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\code}[1]{\texttt{#1}\xspace}$
$\newcommand{\FIXME}[1]{\textcolor{red}{#1}}$
$\newcommand{\gaia}{\textit{Gaia}\xspace}$
$\newcommand{\Gaia}{\gaia}$
$\newcommand{\unit}[1]{\ensuremath{\mathrm{ #1}}\xspace}$
$\newcommand{\feh}{\unit{[Fe/H]}}$
$\newcommand{\mgh}{\unit{[Mg/H]}}$
$\newcommand{\sih}{\unit{[Si/H]}}$
$\newcommand{\cfe}{\unit{[C/Fe]}}$
$\newcommand{\ch}{\unit{[C/H]}}$
$\newcommand{\mh}{\unit{[M/H]}}$
$\newcommand{\zzsun}{\ensuremath{Z/Z_\odot}\xspace}$
$\newcommand{\logzzsun}{\ensuremath{\log Z/Z_\odot}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\Teff}{\teff}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\alphafe}{\unit{[\alpha/Fe]}}$
$\newcommand{\vhel}{\mbox{v_{\mathrm{hel}}}}$
$\newcommand{\vgsr}{\mbox{v_{\mathrm{gsr}}}}$
$\newcommand{\pmragsr}{\mbox{\mu_{\alpha,\mathrm{gsr}}}}$
$\newcommand{\pmdecgsr}{\mbox{\mu_{\delta,\mathrm{gsr}}}}$
$\newcommand{\sigmav}{\mbox{{\sigma_v}}}$
$\newcommand{\sigmafeh}{\mbox{\sigma_{\mathrm{[Fe/H]}}}}$
$\newcommand{\kms}{\unit{km s^{-1}}}$
$\newcommand{\masyr}{\unit{mas yr^{-1}}}$
$\newcommand{\msun}{\unit{M_\odot}}$
$\newcommand{\lsun}{\unit{L_\odot}}$
$\newcommand{\umpstarsdss}{SDSS~J0715-7334\xspace}$
$\newcommand{\umpstar}{J0715-7334\xspace}$
$\newcommand{\umpstartmass}{2MASS~J07153858-7334530\xspace}$
$\newcommand{\umpstargaia}{Gaia~DR3~5262850721755411072\xspace}$
$\newcommand{\umpstarsdssid}{SDSS\_ID~95803549\xspace}$
$\newcommand{\cdstar}{CD-38~245\xspace}$
$\newcommand{\caffaustarsdss}{SDSS J1029+1729\xspace}$
$\newcommand{\caffaustar}{J1029+1729\xspace}$
$\newcommand{\kellerstar}{SMSS J0313-6708\xspace}$
$\newcommand{\lmcstar}{LMC-119\xspace}$
$\newcommand{\sclstar}{Scl-07\xspace}$
$\newcommand{\asstar}{AS0039\xspace}$
$\newcommand{\pristinestar}{Pristine\_221}$
$\newcommand{\thefigure}{Extended Data Figure \arabic{figure}}$
$\newcommand{\thetable}{Extended Data Table \arabic{table}}$</div>



<div id="title">

# A nearly pristine star from the Large Magellanic Cloud

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.21643-b31b1b.svg)](https://arxiv.org/abs/2509.21643)<mark>Appeared on: 2025-09-29</mark> -  _18 pages, 9 figures, 1 table, submitted_

</div>
<div id="authors">

A. P. Ji, et al. -- incl., <mark>P. Eitner</mark>, <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** The first stars formed out of pristine gas, causing them to be so massive that none are expected to have survived until today Bromm2009, Klessen2023 .  If their direct descendants were sufficiently low-mass stars, they could exist today and would be recognizable by having the lowest metallicity (abundance of elements heavier than helium) Frebel2015 . The lowest metallicity star currently known is a star in the thick disk of the Milky Way with total metallicity $Z < 1.4 \times 10^{-6}$ ( $\logzzsun < -4.0$ ) Caffau2011, Lagae2023, Caffau2024 . While other stars with lower iron abundance have been discovered Christlieb2002, Frebel2005, Keller2014 , they have high carbon abundances and thus higher total metallicities ( $\logzzsun > -3$ ). Here we present the discovery and detailed chemical analysis of the most metal-poor star yet found: the red giant star SDSS J0715-7334 with ultra-low abundances of both iron and carbon ( $\mbox{[Fe/H]} = -4.3$ , $\mbox{[C/Fe]} < -0.2$ ), resulting in total metallicity $Z < 7.8 \times 10^{-7}$ ( $\logzzsun < -4.3$ ). This star has the most pristine composition of any object known in the universe. The star's orbit indicates that it originates from the halo of the Large Magellanic Cloud. Its detailed chemical composition implies a supernova progenitor with initial mass of 30 solar masses Heger2010 . Current models of low-mass star formation can explain the existence of SDSS J0715-7334 only if dust cooling was already able to operate at the time of its formation Schneider2003, Schneider2012 . SDSS J0715-7334 is over ten times more metal-poor than the most metal-poor high-redshift galaxies found by the James Webb Space Telescope, some of which have been claimed to be potentially metal-free Fujimoto2025, Nakajima2025, Morishita2025 . Substantially deeper observations of high-redshift galaxies would be needed to prove that they are truly pristine galaxies made of metal-free stars and not metal-enriched galaxies composed of second-generation stars like SDSS J0715 $-$ 7334 Katz2023 .

</div>

<div id="div_fig1">

<img src="tmp_2509.21643/./nature_2a_fehch.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Carbon and iron abundances of ultra-metal-poor stars.**$\umpstar$ is shown as a large red star. Black points show a literature sampleAbohalima2018, Sestito2019. Colored points highlight eight other notable stars, with 1D LTE abundances shown as small open symbols and a combination of 1D NLTE, 3D LTE, and 3D NLTE analyses shown as large solid colored symbols. The dashed blue contours indicate an approximate total metallicity assuming $\mbox{[Mg/Fe]}=+0.4$(see Methods).
 (*fig:cfe*)

</div>
<div id="div_fig2">

<img src="tmp_2509.21643/./ump_yz_timeticks.png" alt="Fig3.1" width="50%"/><img src="tmp_2509.21643/./ump_orbit_map.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** **Kinematic properties.**_Top:_
    Past orbit of $\umpstar$ over $4$ Gyr in Galactocentric coordinates, integrated in a potential that includes the gravitational influence of the LMC (solid lines).
    Circular markers are placed every $1$ Gyr.
    For comparison, the past orbit of the LMC itself is shown, along with orbits of the stars $\caffaustar$(confined to the disk) and LMC-119 (closely bound to the LMC).
    The dashed red line shows the unbound orbit of $\umpstar$ in a MW-only potential.
    _Bottom:_ The past orbit of $\umpstar$ and the LMC in Galactic coordinates on-sky, overlaid on the distribution of all stars observed by _Gaia_.
     (*fig:kinematics*)

</div>
<div id="div_fig3">

<img src="tmp_2509.21643/./nature_pop3fit.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Population III Supernova Progenitor Constraints.**_Top:_ Chemical abundance pattern of $\umpstar$. Filled red stars show measured abundances with $1\sigma$ uncertainties; open red stars with arrows are upper limits (treated as hard cutoffs). Sc is treated as an upper limit due to model uncertainties; Cr and Zn are excludedHeger2010. The blue line and caption shows the best-fit model; black lines show other models within $95\%$ confidence, with opacity indicating fit quality.
    _Bottom left:_ Progenitor mass distribution of models weighted using total absolute error (see Methods).
    _Bottom right:_ Explosion energy vs. progenitor mass for the same models; point size scales with fit quality. Best-fit model shown in blue.
     (*fig:popiii*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.21643"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aap}{{Astron. Astrophys.}}$
$\newcommand{\aapr}{{Astron. Astrophys. Rev.}}$
$\newcommand{\aj}{{Astron. J.}}$
$\newcommand{\apj}{{Astrophys. J.}}$
$\newcommand{\apjl}{{Astrophys. J. Lett.}}$
$\newcommand{\apjs}{{Astrophys. J. Supp. Ser.}}$
$\newcommand{\apss}{{Astrophys. Space Sci.}}$
$\newcommand{\araa}{{Annu. Rev. Astron. Astrophys.}}$
$\newcommand{\grl}{{Geophys. Res. Lett.}}$
$\newcommand{\icarus}{{Icarus}}$
$\newcommand{\iaucirc}{{Int. Astron. U. Circ.}}$
$\newcommand{\jgr}{{J. Geophys. Res.}}$
$\newcommand{\mnras}{{Mon. Not. Roy. Astron. Soc.}}$
$\newcommand{\na}{{New Astronomy}}$
$\newcommand{\nat}{{Nature}}$
$\newcommand{\pasj}{{Publ. Astron. Soc. Jap.}}$
$\newcommand{\pasp}{{Publ. Astron. Soc. Pac.}}$
$\newcommand{\planss}{{Plan. Spac. Sci.}}$
$\newcommand{\procspie}{{Proc. of SPIE}}$
$\newcommand{\solphys}{{Solar Phys.}}$
$\newcommand{\ssr}{{Space Sci. Rev.}}$
$\newcommand{\psj}{{Planet. Sci. J.}}$
$\newcommand{\prl}{{Phys. Rev. Lett.}}$
$\newcommand{\bain}{{Bull. Astron. Inst. Netherlands}}$
$\newcommand{\timCom}[1]{\textbf{{\color{red}[#1 - Tim]}}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\dimitriCom}[1]{\textbf{{\color{orange}[#1 - Dimitri]}}}$
$\newcommand{\orange}[1]{\textcolor{orange}{#1}}$
$\newcommand{\MattHCom}[1]{\textbf{{\color{teal}[#1 - Matt H.]}}}$
$\newcommand{\rk}[1]{\textbf{{\color{magenta}[#1 - RK]}}}$</div>



<div id="title">

# The Origins \& Reservoirs of Exocomets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.22541-b31b1b.svg)](https://arxiv.org/abs/2509.22541)<mark>Appeared on: 2025-09-29</mark> -  _Accepted for publication in Space Science Reviews. Chapter written for the the workshop 'Exocomets: Bridging our Understanding of Minor Bodies in Solar and Exoplanetary Systems' (July 2024), at the International Space Science Insitute (ISSI), Bern_

</div>
<div id="authors">

M. Bannister, et al. -- incl., <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** Small bodies exist in distinct populations within their planetary systems.These reservoir populations hold a range of compositions, which to first order are dependent on formation location relative to their star.We provide a general overview of the nature of the reservoirs that source exocomets, from the influence of the stellar environment through planetesimal formation to comparisons with Solar System populations.Once transitioned from a young protoplanetary disc to a debris disc, a star can expect to be rained with exocomets.While exocomets are predominantly detected to date at A-type stars, planetesimals plausibly exist across a range of stellar masses, based on exoplanet abundance, debris disc occurrence and white dwarf infall.

</div>

<div id="div_fig1">

<img src="tmp_2509.22541/./mDustAndWidthVsAge.png" alt="Fig4" width="100%"/>

**Figure 4. -** Dust mass (left) and radius (right) of protoplanetary and debris discs as a function of system age. Protoplanetary discs are represented with squares and debris or evolved discs with circles. The Kuiper belt is represented with a star on the right panel. Debris data from \cite{Matra2025}, and protoplanetary disc data from \citet{Bae2023}. The colour scale indicates the stellar mass in both panels.
     (*fig:DebrisDiscMassAndLocationVsAge*)

</div>
<div id="div_fig2">

<img src="tmp_2509.22541/./ExocometsInterstellar.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** Schematic picture illustrating the approximate location of the different small-body and comet reservoirs in the Solar System.  (*fig:Location_reservoirs*)

</div>
<div id="div_fig3">

<img src="tmp_2509.22541/./Exocomet_Growth.png" alt="Fig2" width="100%"/>

**Figure 2. -** Schematic picture illustrating the growth from dust to planetesimal sizes.
    `Dust' of silicate- and organics-rich grains are shown schematically; these grow to fractal dust structures via sticking collisions.
    After compactifying and further growth they reach planetesimal sizes.
    Credits: a) Amara/Wikipedia (CC), b) via Wikimedia Commons, c) adapted from Wikipedia Commons, and d) cold classical Arrokoth (NASA). (*fig:Dust2planetesimals*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.22541"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

124  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
